In [ ]:
# IMPORTANT: The parameters below are set only for running this notebook independently. 
# When executing the full Ploomber pipeline, these values will be overridden by the settings in `pipeline.yaml`. 
# Any modifications made here will not persist when running the pipeline.
upstream = None
COUNTRY =  'ARM' # Code of the Country
product = {'data': f'../data/processed/{COUNTRY}/partnership_transfers.xlsx'}  # Path to save the final data product (stored under the 'data' key)
data_source = 'data/raw/insight-dct-partnerships/insight-dct-partnerships-analysis-cube.xlsx'  # Path to the source data directory

This Notebook summarizes the direct cash transfers (DCT) to partners, including total transfer amounts and goal areas (copy of **Partnership Transfers** in 01_main).

In [ ]:
import re
from pathlib import Path
import pandas as pd
import unicef_cpe as cpe

from unicef_cpe.config import PROJ_ROOT



country_map = {k:v for k,v in cpe.utils.get_ecaro_countries_mapping(priority=False).items() if k in COUNTRY}
country_code_map = {v:k for k,v in country_map.items() }

**Partnership Transfers**

In [ ]:
file_path = PROJ_ROOT / data_source
df_partnerships = pd.read_excel(file_path, sheet_name='by BA, Partner & FR', skiprows=6, skipfooter=1)
df_partnerships.rename(lambda x: x.lower().replace(' ', '_'), axis=1, inplace=True)
print('Shape:', df_partnerships.shape)
print(df_partnerships.head())

In [ ]:
df_partnerships.isna().sum()

In [ ]:
for column in ('vendor', 'fr_start_year', 'fr_end_year', 'budget_business_area'):
    df_partnerships[column] = df_partnerships[column].ffill()

for column in ('fr_start_year', 'fr_end_year'):
    df_partnerships[column] = df_partnerships[column].astype(int)

print('Shape:', df_partnerships.shape)
print(df_partnerships.head())

In [ ]:
mapping = cpe.utils.get_ecaro_countries_mapping(keys='code', values='iso')
df_partnerships['country'] = df_partnerships['budget_business_area'].str.split(' - ').str.get(-1).replace(mapping)
df_partnerships = df_partnerships.loc[df_partnerships['country'].isin(mapping.values())].copy()
assert not set(df_partnerships['country']) - set(mapping.values())

print('Shape:', df_partnerships.shape)
print(df_partnerships.head())

In [ ]:
df_partnerships = df_partnerships[df_partnerships['country'].eq(COUNTRY)].copy()

In [ ]:
output_path = Path(product['data'])
output_path.parent.mkdir(parents=True, exist_ok=True)  # Create missing directories
df_partnerships.to_excel(product['data'], index=False)